In [101]:
# PSEUDOCODE

# split data
data = ...
train, validation, test = split(data)
 
# tune model hyperparameters
parameters = ...
for params in parameters:
	model = fit(train, params)
	skill = evaluate(model, validation)
 
# evaluate final model for comparison with other models
model = fit(train)
skill = evaluate(model, test)

NameError: name 'split' is not defined

In [114]:
# Libraries

import os, shutil, keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras import layers
from keras import models

from keras.models import Sequential
from keras.layers import Conv2D, Convolution2D, MaxPooling2D, Flatten, Dense, Dropout

from keras.preprocessing.image import ImageDataGenerator

In [115]:
#directories with training, validation, test cat/dog pics

train_dir = os.path.abspath("/Users/Nadia/Desktop/FinalProject490/train")
validation_dir = os.path.abspath("/Users/Nadia/Desktop/FinalProject490/validation")
test_dir = os.path.abspath("/Users/Nadia/Desktop/FinalProject490/test")

train_rock = os.path.join(train_dir, "train_rock")
train_paper = os.path.join(train_dir, "train_paper")
train_scissors = os.path.join(train_dir, "train_scissors")

validation_rock = os.path.join(validation_dir, "validation_rock")
validation_paper = os.path.join(validation_dir, "validation_paper")
validation_scissors = os.path.join(validation_dir, "validation_scissors")

test_rock = os.path.join(test_dir, "test_rock")
test_paper = os.path.join(test_dir, "test_paper")
test_scissors = os.path.join(test_dir, "test_scissors")

In [117]:
# Model

model = models.Sequential()

model.add(Conv2D(16, (3, 3), activation='relu',
input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation = 'softmax'))

In [119]:
# Save model & summary

model.save('rps_model.h5')
model.summary()

Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_61 (Conv2D)           (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d_61 (MaxPooling (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_62 (Conv2D)           (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_62 (MaxPooling (None, 36, 36, 32)        0         
_________________________________________________________________
flatten_33 (Flatten)         (None, 41472)             0         
_________________________________________________________________
dense_49 (Dense)             (None, 128)               5308544   
_________________________________________________________________
dropout_17 (Dropout)         (None, 128)             

In [124]:
# data-augmentation generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,  #target directory
    target_size = (150, 150),
    batch_size = 16,
    shuffle = True,
    class_mode = 'categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_dir,  #target directory
    target_size = (150, 150),
    batch_size = 16,
    shuffle = True,
    class_mode = 'categorical')




Found 1531 images belonging to 3 classes.
Found 437 images belonging to 3 classes.


In [125]:
train_generator.class_indices

{'train_paper': 0, 'train_rock': 1, 'train_scissors': 2}

In [126]:
# Compile model for training
from keras import optimizers

model.compile(loss='categorical_crossentropy', 
    optimizer =keras.optimizers.Adam(learning_rate=1e-4, beta_1=0.9, beta_2=0.999, amsgrad=False),
    metrics=['acc'])

In [ ]:
# Run model
history = model.fit_generator(
    train_generator,
    steps_per_epoch = 100,
    epochs = 30,
    validation_data = validation_generator,
    validation_steps = 50)

Epoch 1/30
100/100 [==============================] - 97s 966ms/step - loss: 1.1125 - acc: 0.3693 - val_loss: 1.0120 - val_acc: 0.5577
Epoch 2/30
100/100 [==============================] - 74s 742ms/step - loss: 1.0509 - acc: 0.4408 - val_loss: 1.0171 - val_acc: 0.5501
Epoch 3/30
100/100 [==============================] - 46s 459ms/step - loss: 1.0147 - acc: 0.5028 - val_loss: 0.8866 - val_acc: 0.7442
Epoch 4/30
100/100 [==============================] - 33s 326ms/step - loss: 0.9751 - acc: 0.5266 - val_loss: 0.7426 - val_acc: 0.8046
Epoch 5/30
100/100 [==============================] - 34s 340ms/step - loss: 0.9605 - acc: 0.5354 - val_loss: 0.7222 - val_acc: 0.8758
Epoch 6/30
100/100 [==============================] - 35s 350ms/step - loss: 0.9172 - acc: 0.5698 - val_loss: 0.7223 - val_acc: 0.8085
Epoch 7/30
100/100 [==============================] - 69s 687ms/step - loss: 0.8818 - acc: 0.5994 - val_loss: 0.6309 - val_acc: 0.9190
Epoch 8/30
100/100 [==============================] - 7

In [ ]:
# display acc & loss curves during training and validation
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
# evaluate model
acc = model.evaluate_generator(validation_generator, steps=100, verbose=0)
print('> %.3f' % (acc * 100.0))